In [ ]:
import os
import sys
import glob
import numpy as np
import pandas as pd
import datetime as dt
from dateutil.parser import parse
import datefinder
import dateinfer
import logging

In [ ]:
def set_logger(level=logging.INFO, name=__name__):
  try: logger = logging.getLogger(__class__.__qualname__)
  except: logger = logging.getLogger(name)
  logger.setLevel(level)
  handler = logging.FileHandler('log_file.log')
  formatter = logging.Formatter('%(asctime)s : %(name)s  : %(funcName)s : %(levelname)s : %(message)s')
  handler.setFormatter(formatter)
  logger.addHandler(handler)
  return logger

In [ ]:
dict_dir = "summaries.csv"
summaries = pd.read_csv(dict_dir)[['CIK', 'FILING_DATE', 'ACC_NUM', 'FORM_TYPE' ,'CoName']]
# get cik.csv
a = pd.read_csv('cik.csv')
# prettier
a.drop(a.columns[0], axis=1, inplace=True)
# limit exchange to NYSE or NASDAQ
# due to duplicated cik are common in OTC, we need to limit exchange first
a.drop(a.index[(a['exchange'] != 'NYSE')&(a['exchange'] != 'Nasdaq')], inplace=True)
# drop all duplicates
a.drop_duplicates(subset='cik', keep='first', inplace=True)
a.drop_duplicates(subset='name', keep='first', inplace=True)
a.drop_duplicates(subset='ticker', keep='first', inplace=True)
# column cik to index
a.set_index('cik', inplace=True)
ticker = a
# add col 'ticker', 'exchange' to summaries
def findticker(cik):
    try: result = ticker.at[cik, 'ticker']
    except: return
    return result
def findexchange(cik):
    try: result = ticker.at[cik, 'exchange']
    except: return
    return result
summaries['TICKER'] = summaries['CIK'].map(findticker)
summaries['EXCHANGE'] = summaries['CIK'].map(findexchange)

In [ ]:
def find_path_from_date(filing_date):
    filing_date = str(filing_date)
    year = filing_date[:4]
    # month to quarter
    month = int(filing_date[4:6])
    quarter = (month-1) // 3 + 1
    quarter = 'QTR' + str(quarter)
    path = os.path.join(PATH, year, quarter)
    return path
path = pd.DataFrame(columns=['acc', 'filename', 'path'])
path['acc'] = summaries['ACC_NUM']
path['filename'] = summaries['FILING_DATE'].map(str) + '_' + summaries['FORM_TYPE'] + '_edgar_data_'  + summaries['CIK'].map(str) + '_' + summaries['ACC_NUM'].map(str) + '.txt'
path['path'] = summaries['FILING_DATE'].map(find_path_from_date) +os.sep+path['filename']
path['acc'].map(str)
path.set_index('acc', inplace=True)

In [ ]:
# innocent = ACC_NUM이 중복되지 않은 리스트
innocent = summaries.drop_duplicates(subset=['ACC_NUM'], keep=False)
print(len(innocent))
inn_cik =  innocent['CIK'].value_counts()
inn_co =  innocent['CoName'].value_counts()
inn_co
print(len(inn_cik), len(inn_co))
# get duplicated acc num
duplicated = summaries[summaries['ACC_NUM'].duplicated(keep=False)]
counts = duplicated['ACC_NUM'].value_counts()
coname = duplicated['CoName'].value_counts()
cik = duplicated['CIK'].value_counts()
acc = counts.index
print(coname)
print(cik)
summaries['CoName']